In [1]:
from ipynb.fs.full.utils import *
import numpy as np
import h5py
import pandas as pd
import sys
import illustris_python.illustris_python as il
import matplotlib.pyplot as plt
import os
from multiprocessing import Pool
from itertools import repeat
import tqdm

SOLAR_METALLICITY = 0.0127

In [2]:
def calculate_age(expansionFactor, hubble_param, omega0, omegaLambda):
    hubble_time = 9.778 / hubble_param  # Gyr

    factor1 = 2.0 / 3.0 / np.sqrt(omegaLambda)
    term1 = np.sqrt(omegaLambda / omega0) * expansionFactor ** 1.5
    term2 = np.sqrt(1 + omegaLambda / omega0 * expansionFactor ** 3)
    factor2 = np.log(term1 + term2)

    time = factor1 * factor2 * hubble_time

    return time


def get_mass_weighted_age_met(subhalo_id, snapshot, sim_dir):
    basePath = '/home/jovyan/Data/Sims/IllustrisTNG/{}/'.format(sim_dir)
    (scaling_factor, hubble_param,
        boxsize, omega0, omegaLambda) = get_snapshot_values(basePath, snapshot)

    mw_age, mw_met = 0, 0
    fields = ['ParticleIDs', 'GFM_StellarFormationTime', 'GFM_Metallicity', 'Masses']   
    stars = il.snapshot.loadSubhalo(basePath, snapshot, subhalo_id,
                                    'stars', fields=fields)
    
    if stars['count'] > 0:
        particle_ids = stars['ParticleIDs']
        formation_time = stars['GFM_StellarFormationTime']
        
        stars_mask = np.where(formation_time > 0)[0]
        masses = stars['Masses'][stars_mask]
        metallicity = stars['GFM_Metallicity'][stars_mask] / SOLAR_METALLICITY
        formation_time = formation_time[stars_mask]
        
        weighted_metallicity = np.sum(np.multiply(masses, metallicity))
        mw_met = np.divide(weighted_metallicity, np.sum(masses))
        
        stellar_ages = (calculate_age(scaling_factor, hubble_param, omega0, omegaLambda) - 
                        calculate_age(formation_time, hubble_param, omega0, omegaLambda))
        weighted_age = np.sum(np.multiply(masses, stellar_ages))
        mw_age = np.divide(weighted_age, np.sum(masses))
    return (mw_met, mw_age)



In [3]:
def calc_met_age_sim(sim_dir):
    basePath = '/home/jovyan/Data/Sims/IllustrisTNG/{}/'.format(sim_dir)
    met_age_dir = 'MetAge/IllustrisTNG/{}'.format(sim_dir)
    if not os.path.exists(met_age_dir):
        os.makedirs(met_age_dir)

    for snapshot in range(29, 34):
        (scaling_factor, hubble_param,
            boxsize, omega0, omegaLambda) = get_snapshot_values(basePath, snapshot)
        tng = get_catalog_data(basePath, snapshot, scaling_factor, hubble_param)
        galaxy_ids = list(tng.GalaxyID)

        pool = Pool(processes=4)
        results = list(tqdm.tqdm(pool.starmap(get_mass_weighted_age_met, 
                                              zip(galaxy_ids, repeat(snapshot), repeat(sim_dir))), 
                                 total=len(galaxy_ids)))

        pool.close()
        pool.join()

        met, age = map(list, zip(*results))
        met_age_dict = {'GalaxyID': galaxy_ids, 'Metallicity': met, 'Age': age}
        df = pd.DataFrame(met_age_dict)

        save_path = os.path.join(met_age_dir, 'met_age_%03d.hdf5' % snapshot)
        df.to_hdf(save_path, '/data')

